## Lexical Diversity Features

In [2]:
from collections import Counter, defaultdict
from math import sqrt, log
from nltk import word_tokenize
import numpy as np
import pandas as pd
from tqdm import tqdm
import os

In [3]:
def ttr(text):
    tok_text = word_tokenize(text)
    return len(set(tok_text)) / len(tok_text)

In [4]:
def root_ttr(text):
    return sqrt(ttr(text))

In [5]:
def corrected_ttr(text):
    tok_text = word_tokenize(text)
    return sqrt(len(set(tok_text)) / (2 * len(tok_text)))

In [6]:
def log_ttr(text):
    tok_text = word_tokenize(text)
    if log(len(tok_text),2) == 0:
        print(text)
        return 0
    return log(len(set(tok_text)),2) / log(len(tok_text),2)

In [7]:
def uber_index(text):
    tok_text = word_tokenize(text)
    if log(len(tok_text),2) != 0 and log(len(set(tok_text)),2) != 0:
        return (log(len(tok_text),2) ** 2) / (log(len(set(tok_text)),2) / log(len(tok_text),2))
    else:
        return 0

In [8]:
def yule_s_k(text):
    tok_text = word_tokenize(text)
    token_counter = Counter(tok.upper() for tok in tok_text)
    m1 = sum(token_counter.values())
    m2 = sum([freq ** 2 for freq in token_counter.values()])
    if m2-m1 != 0:
        i = (m1*m1) / (m2-m1)
        k = 10000 / i
        return k

In [9]:
#Copyright 2017 John Frens
#
#Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:
#
#The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.
#
#THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

import string

# Global trandform for removing punctuation from words
remove_punctuation = str.maketrans('', '', string.punctuation)

# MTLD internal implementation
def mtld_calc(word_array, ttr_threshold):
    current_ttr = 1.0
    token_count = 0
    type_count = 0
    types = set()
    factors = 0.0
    
    for token in word_array:
        token = token.translate(remove_punctuation).lower() # trim punctuation, make lowercase
        token_count += 1
        if token not in types:
            type_count +=1
            types.add(token)
        current_ttr = type_count / token_count
        if current_ttr <= ttr_threshold:
            factors += 1
            token_count = 0
            type_count = 0
            types = set()
            current_ttr = 1.0
    
    excess = 1.0 - current_ttr
    excess_val = 1.0 - ttr_threshold
    factors += excess / excess_val
    if factors != 0:
        return len(word_array) / factors
    return -1

# MTLD implementation
def mtld(word_array, ttr_threshold=0.72):
    word_array = word_tokenize(word_array)
    
    return (mtld_calc(word_array, ttr_threshold) + mtld_calc(word_array[::-1], ttr_threshold)) / 2


# HD-D internals

# x! = x(x-1)(x-2)...(1)
def factorial(x):
    if x <= 1:
        return 1
    else:
        return x * factorial(x - 1)

# n choose r = n(n-1)(n-2)...(n-r+1)/(r!)
def combination(n, r):
    r_fact = factorial(r)
    numerator = 1.0
    num = n-r+1.0
    while num < n+1.0:
        numerator *= num
        num += 1.0
    return numerator / r_fact

# hypergeometric probability: the probability that an n-trial hypergeometric experiment results 
#  in exactly x successes, when the population consists of N items, k of which are classified as successes.
#  (here, population = N, population_successes = k, sample = n, sample_successes = x)
#  h(x; N, n, k) = [ kCx ] * [ N-kCn-x ] / [ NCn ]
def hypergeometric(population, population_successes, sample, sample_successes):
    return (combination(population_successes, sample_successes) *\
            combination(population - population_successes, sample - sample_successes)) /\
            combination(population, sample)
    
# HD-D implementation
def hdd(word_array, sample_size=42.0):
    word_array = word_tokenize(word_array)
    
    # Create a dictionary of counts for each type
    type_counts = {}
    for token in word_array:
        token = token.translate(remove_punctuation).lower() # trim punctuation, make lowercase
        if token in type_counts:
            type_counts[token] += 1.0
        else:
            type_counts[token] = 1.0
    # Sum the contribution of each token - "If the sample size is 42, the mean contribution of any given
    #  type is 1/42 multiplied by the percentage of combinations in which the type would be found." (McCarthy & Jarvis 2010)
    hdd_value = 0.0
    for token_type in type_counts.keys():
        contribution = (1.0 - hypergeometric(len(word_array), sample_size, type_counts[token_type], 0.0)) / sample_size
        hdd_value += contribution

    return hdd_value

In [10]:
LD_feature_functions = [ttr, root_ttr, corrected_ttr, log_ttr, uber_index, yule_s_k, mtld, hdd]

In [11]:
input_files = os.listdir("/home/soeren/Documents/TextDifficultyAssessmentGerman/datasets/RandomText/FeatureSet1_BF")
input_files = ["01_Preprocessing_df.csv"]

def save_df(path,df):
    df.to_csv(path,sep="|",index=False)

for input_file in input_files:
    df = pd.read_csv("datasets/RandomText/FeatureSet1_BF/" + input_file, sep="|", lineterminator = '\n')

    for function in tqdm(LD_feature_functions):
        df["LD-"+function.__name__]  = [function(text) for text in df["Text"]]

    filename = "datasets/RandomText/FeatureSet2_LD/" + input_file
    save_df(filename, df)

100%|██████████| 8/8 [01:05<00:00,  8.19s/it]


In [43]:
df.head()

,Text,newLevel,RFTagger,parsedText,preprocessedText,BF-average_sentence_length,BF-average_syllable_word,BF-Flesch_Reading_Ease_Score,BF-Flesch_Kincaid_readability,BF-percentag_hard_words,...,BF-average_character_per_word,BF-text_length,LD-ttr,LD-root_ttr,LD-corrected_ttr,LD-log_ttr,LD-uber_index,LD-yule_s_k,LD-mtld,LD-hdd
0,"Deine Frau und Babys geht nach Hause, oder Hos...",A,"[[['Deine', 'PRO', 'Poss', 'Attr', '-', 'Nom',...",['(ROOT (CS (S (NP (ART Deine) (CNP (NN Frau) ...,"[[['Deine', 'PRO', 'Poss', 'Attr', '-', 'Nom',...",8.250,1.581818,64.639432,7.281818,0.109091,...,4.800000,66,0.803030,0.896120,0.633652,0.947642,38.553281,142.332415,66.880000,0.768626
1,Nach der Kneipe ich befahrt den junge Mann zu ...,A,"[[['Nach', 'APPR', 'Dat'], ['der', 'ART', 'Def...",['(ROOT (NUR (PP (APPR Nach) (ART der) (NN Kne...,"[[['Nach', 'APPR', 'Dat'], ['der', 'ART', 'Def...",10.750,1.565789,63.457961,8.089474,0.078947,...,4.605263,86,0.827586,0.909718,0.643268,0.957625,43.348401,89.840137,111.543158,0.839343
2,"Lieber Ingo, Wie gehst dir? Wenn ich nur die b...",A,"[[['Lieber', 'N', 'Reg', 'Nom', 'Sg', 'Masc'],...",['(ROOT (S (S (ADJD Lieber) (NP (NE Ingo))) ($...,"[[['Lieber', 'N', 'Reg', 'Nom', 'Sg', 'Masc'],...",8.125,1.642857,59.602411,7.964286,0.142857,...,4.785714,65,0.846154,0.919866,0.650444,0.959981,37.780860,80.473373,84.500000,0.819016
3,Er ist nicht verheiratet und hat keine Kinder....,A,"[[['Er', 'PRO', 'Pers', 'Subst', '3', 'Nom', '...",['(ROOT (CS (S (PPER Er) (VAFIN ist) (VP (PTKN...,"[[['Er', 'PRO', 'Pers', 'Subst', '3', 'Nom', '...",10.875,1.597403,60.656615,8.518831,0.129870,...,4.857143,87,0.758621,0.870988,0.615882,0.938142,44.248667,132.117849,62.929054,0.790037
4,Du wahrscheinlich hast kein Qualifikation für ...,A,"[[['Du', 'PRO', 'Pers', 'Subst', '2', 'Nom', '...",['(ROOT (S (AP (PPER Du) (ADJD wahrscheinlich)...,"[[['Du', 'PRO', 'Pers', 'Subst', '2', 'Nom', '...",10.625,1.583333,62.100625,8.250000,0.083333,...,4.513889,85,0.735632,0.857690,0.606478,0.931251,44.576064,150.614348,42.270204,0.774356
